Importing required libraries

In [1]:
import matplotlib
import warnings
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.utils.np_utils import to_categorical
from tomatoleaf.lenet import LeNet
from imutils import paths
import numpy as np
import random
import cv2
import os
import time

Accessing dataset and defining parameters

In [2]:
resolution = 75 
dataSet = r'C:/Users/Admin/Documents/AI-PROJECT/New folder/plantvillage'
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
height = resolution
width = resolution
modelName = 'LeafDisease75x75.h5'

Printing total number of classes present

In [3]:
path = dataSet
print(path)
cnames = os.listdir(path)
totClass = len(cnames)
print(f'Total Classes = {totClass}')
print(f'Available Classes : {cnames}')

C:/Users/Admin/Documents/AI-PROJECT/New folder/plantvillage
Total Classes = 10
Available Classes : ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']


Accessing image paths

In [5]:
data = []
labels = []
imagePaths = sorted(list(paths.list_images(dataSet)))
#print(len(imagePaths))
random.shuffle(imagePaths)
imageFailed = 0

Resizing the shape of image and appending images and their labels to respective lists

In [6]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (height, width))
    image = img_to_array(image)
    lbl = imagePath.split(os.path.sep)[-2]
    label = cnames.index(lbl);
    #print(label)
    data.append(image)
    labels.append(label)
             

In [ ]:
print(data[1])

In [ ]:
print(labels)

Performing train test split

In [7]:
data = np.array(data, dtype="float") / 255.0
print(data[1])
labels = np.array(labels)
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)


[[[0.50588235 0.47843137 0.50588235]
  [0.58823529 0.56078431 0.58823529]
  [0.55686275 0.52941176 0.55686275]
  ...
  [0.34901961 0.31764706 0.34509804]
  [0.48235294 0.45098039 0.47843137]
  [0.78823529 0.75294118 0.78039216]]

 [[0.48235294 0.45490196 0.48235294]
  [0.54117647 0.51372549 0.54117647]
  [0.71372549 0.68627451 0.71372549]
  ...
  [0.5372549  0.50588235 0.53333333]
  [0.54901961 0.51764706 0.54509804]
  [0.50980392 0.47843137 0.50980392]]

 [[0.56862745 0.54117647 0.56862745]
  [0.56078431 0.53333333 0.56078431]
  [0.68235294 0.65490196 0.68235294]
  ...
  [0.51764706 0.48627451 0.51372549]
  [0.61960784 0.58823529 0.61568627]
  [0.58823529 0.55686275 0.58431373]]

 ...

 [[0.49019608 0.4627451  0.52156863]
  [0.48235294 0.45490196 0.51372549]
  [0.45098039 0.42352941 0.48235294]
  ...
  [0.49803922 0.47058824 0.52941176]
  [0.46666667 0.43921569 0.49803922]
  [0.42745098 0.4        0.45882353]]

 [[0.44705882 0.41960784 0.47843137]
  [0.45490196 0.42745098 0.48627451]


In [ ]:
trainX.shape

Converting it to Categorical Data

In [ ]:
trainY = to_categorical(trainY, num_classes=totClass)
testY = to_categorical(testY, num_classes=totClass)
BS=40

In [ ]:
print(testY)

Performing Image Augmentation

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
horizontal_flip=True, fill_mode="nearest")


Implementing LeNet and adam optimiser

In [ ]:
model = LeNet.build(width=width, height=height, depth=3, classes=totClass)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

Loss and Accuracy Estimation

In [27]:
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS, verbose=1)

254/254 [==============================] - 207s 816ms/step - batch: 126.5000 - size: 39.9606 - loss: 0.1204 - accuracy: 0.9596 - val_loss: 2.1396 - val_accuracy: 0.6900
Epoch 24/25
254/254 [==============================] - 212s 835ms/step - batch: 126.5000 - size: 39.9606 - loss: 0.1221 - accuracy: 0.9567 - val_loss: 2.9364 - val_accuracy: 0.6385
Epoch 25/25
254/254 [==============================] - 235s 927ms/step - batch: 126.5000 - size: 39.9606 - loss: 0.1341 - accuracy: 0.9531 - val_loss: 1.7016 - val_accuracy: 0.6915


Saving results into .h5 file

In [ ]:
model.save("LeafDisease.h5")

In [ ]:
Perfoming testing

In [8]:
from tensorflow.keras.models import load_model
import imutils

In [21]:
a=testX[400]
orig = a.copy()
a = np.expand_dims(a, axis=0)

In [10]:
mdl = r"C:/Users/Admin/Documents/AI-PROJECT/Tomato_Leaf_Disease_Detection-master/LeafDisease.h5"


In [16]:
model = load_model(mdl)
(zero, one,two, three,four,five,six,seven, eight,nine) = model.predict(a)[0]
prob = [zero, one,two, three,four,five,six,seven, eight,nine] 


In [22]:
maxProb = max(prob)
maxIndex = prob.index(maxProb)
label = cnames[maxIndex]
proba = maxProb
for i in range(0,totClass):
    print(f'{cnames[i]} : {prob[i]}')
    
label = "{}: {:.2f}%".format(label, proba * 100)

print('\n\n'+str(label))

output = imutils.resize(orig, width=500)
cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
0.9, (0, 255, 0), 1)

cv2.imshow("Output", output)
cv2.waitKey(0)


Tomato___Bacterial_spot : 0.010097220540046692
Tomato___Early_blight : 1.3905315654483275e-06
Tomato___healthy : 0.014484831131994724
Tomato___Late_blight : 0.9748519659042358
Tomato___Leaf_Mold : 1.8070285534577124e-07
Tomato___Septoria_leaf_spot : 3.041324589503347e-06
Tomato___Spider_mites Two-spotted_spider_mite : 6.84829501551576e-06
Tomato___Target_Spot : 1.938763034559088e-06
Tomato___Tomato_mosaic_virus : 3.2762600965519084e-10
Tomato___Tomato_Yellow_Leaf_Curl_Virus : 0.0005524170701391995


Tomato___Late_blight: 97.49%


-1